# Genetic optimisation of SVC classifier for the SDSS data

## Load data

In [ ]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals
import warnings
import helpers
from helpers import DataSet
import matplotlib as mpl
import os
import random

# Common imports
import pandas as pd
import numpy as np
import seaborn as sns

# Imports for ML
from sklearn.ensemble import RandomForestClassifier, VotingClassifier, BaggingClassifier, GradientBoostingClassifier, AdaBoostClassifier, ExtraTreesClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier, PassiveAggressiveClassifier, RidgeClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import RobustScaler, StandardScaler, MaxAbsScaler
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.tree import DecisionTreeClassifier
from sklearn.neural_network import MLPClassifier, 
from sklearn.naive_bayes import GaussianNB, MultinomialNB
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis
from sklearn.decomposition import PCA

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)
sns.set_style('whitegrid')

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "results"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)

# Helper functioins and structures
# Ignore useless warnings (see SciPy issue #5998)
warnings.filterwarnings(action="ignore", message="^internal gelsd")

DATA_PATH = "Skyserver_SQL2_27_2018 6_51_39 PM.csv"
RESULTS_FOLDER = "results"

# We load the data. Those that have nothing to do with the features of the objects are ignored.
sdss_data = pd.read_csv(DATA_PATH)

# We have a general look at the features
sdss_data.head(3)

print(sdss_data.columns.values)
sdss_data.drop(['objid', 'run', 'rerun', 'camcol', 'field',
                'specobjid', 'fiberid', 'mjd', 'plate'], axis=1, inplace=True)
sdss_data.head(1)

sdss_df_fe = sdss_data

# Principal Component Analysis
pca = PCA(n_components=3)
ugriz = pca.fit_transform(sdss_df_fe[['u', 'g', 'r', 'i', 'z']])

# update dataframe 
sdss_df_fe = pd.concat((sdss_df_fe, pd.DataFrame(ugriz)), axis=1)
sdss_df_fe.rename({0: 'PCA_1', 1: 'PCA_2', 2: 'PCA_3'}, axis=1, inplace = True)
sdss_df_fe.drop(['u', 'g', 'r', 'i', 'z'], axis=1, inplace=True)
sdss_df_fe.head()

X = sdss_data.drop(['class'], axis=1)
y = sdss_data['class']

class_names = ["GALAXY", "QSO", "STAR"]

std_scaler = StandardScaler()
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

X_test = std_scaler.fit_transform(X_test.astype(np.float64))
X_train = std_scaler.fit_transform(X_train.astype(np.float64))

## Optimize

In [6]:
from evolutionary_search import EvolutionaryAlgorithmSearchCV

paramgrid = {"hidden_layer_sizes": [(100, ),
                                    (200,),
                                    (100,50),
                                    (200,50,100),
                                    (100,100,100),
                                    (50,50,50),
                                    (200,200,200,200),
                                    (100,50,100,200)],
             "activation": ["relu", "identity", "logistic", "tanh"],
             "solver": ["adam", "lbfs", "sgd"],
             "alpha": np.logspace(-5, 1, base=10, num=25),
             "batch_size": np.logspace(0,4, base=10, num=25, dtype=int),
             "learning_rate": ["constant", "invscaling", "adaptive"],
             "learning_rate_init": np.logspace(-4, 1, base=10, num=25),
             "power_t": np.logspace(-3, 0, base=10, num=25),
             "max_iter": np.logspace(2, 5, base=10, num=25, dtype=int),
             "random_state": [42],
             "momentum": np.logspace(-3, 0, base=10, num=25),
             "nesterovs_momentum": [True, False],
             "early_stopping": [False, True],
             "beta_1": [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7 ,0.9],
             "beta_2": np.logspace(-1, -0.001, base=10, num=25),
             "epsilon": [1e-08],
             }


random.seed(1)

cv = EvolutionaryAlgorithmSearchCV(estimator=MLPClassifier(),
                                   params=paramgrid,
                                   scoring="accuracy",
                                   cv=StratifiedKFold(n_splits=5),
                                   verbose=3,
                                   population_size=50,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=15,
                                   n_jobs=8,
                                   error_score=0.0)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    result = cv.fit(X_train, y_train)

Types [1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 2, 2, 2] and maxint [7, 3, 2, 24, 24, 2, 24, 24, 24, 0, 24, 1, 1, 9, 24, 0] detected


/home/ljosfer/anaconda3/envs/deep/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/ljosfer/anaconda3/envs/deep/lib/python3.6/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


[CV] hidden_layer_sizes=(100, 50), activation=relu, solver=lbfs, alpha=5.623413251903491e-05, batch_size=316, learning_rate=invscaling, learning_rate_init=0.1333521432163324, power_t=0.31622776601683794, max_iter=3162, random_state=42, momentum=0.0023713737056616554, nesterovs_momentum=False, early_stopping=False, beta_1=0.5, beta_2=0.3476362047879868, epsilon=1e-08 
[CV]  hidden_layer_sizes=(100, 50), activation=relu, solver=lbfs, alpha=5.623413251903491e-05, batch_size=316, learning_rate=invscaling, learning_rate_init=0.1333521432163324, power_t=0.31622776601683794, max_iter=3162, random_state=42, momentum=0.0023713737056616554, nesterovs_momentum=False, early_stopping=False, beta_1=0.5, beta_2=0.3476362047879868, epsilon=1e-08, score=0.0, total=   0.0s
[CV] hidden_layer_sizes=(100, 50), activation=relu, solver=lbfs, alpha=5.623413251903491e-05, batch_size=316, learning_rate=invscaling, learning_rate_init=0.1333521432163324, power_t=0.31622776601683794, max_iter=3162, random_state=42

[CV]  hidden_layer_sizes=(100, 50), activation=relu, solver=lbfs, alpha=5.623413251903491e-05, batch_size=316, learning_rate=invscaling, learning_rate_init=0.1333521432163324, power_t=0.31622776601683794, max_iter=3162, random_state=42, momentum=0.0023713737056616554, nesterovs_momentum=False, early_stopping=False, beta_1=0.5, beta_2=0.3476362047879868, epsilon=1e-08, score=0.0, total=   0.0s
[CV]  hidden_layer_sizes=(200, 50, 100), activation=tanh, solver=lbfs, alpha=0.1778279410038923, batch_size=14, learning_rate=invscaling, learning_rate_init=0.0028729848333536655, power_t=0.4216965034285822, max_iter=749, random_state=42, momentum=0.01333521432163324, nesterovs_momentum=True, early_stopping=True, beta_1=0.7, beta_2=0.6799860450033224, epsilon=1e-08, score=0.0, total=   0.0s
[CV] hidden_layer_sizes=(200, 50, 100), activation=tanh, solver=lbfs, alpha=0.1778279410038923, batch_size=14, learning_rate=invscaling, learning_rate_init=0.0028729848333536655, power_t=0.4216965034285822, max

[CV]  hidden_layer_sizes=(100, 100, 100), activation=tanh, solver=lbfs, alpha=0.01778279410038923, batch_size=10, learning_rate=invscaling, learning_rate_init=0.00042169650342858224, power_t=0.01, max_iter=74989, random_state=42, momentum=0.23713737056616552, nesterovs_momentum=False, early_stopping=False, beta_1=0.2, beta_2=0.1, epsilon=1e-08, score=0.0, total=   0.0s
[CV] hidden_layer_sizes=(100, 100, 100), activation=tanh, solver=lbfs, alpha=0.01778279410038923, batch_size=10, learning_rate=invscaling, learning_rate_init=0.00042169650342858224, power_t=0.01, max_iter=74989, random_state=42, momentum=0.23713737056616552, nesterovs_momentum=False, early_stopping=False, beta_1=0.2, beta_2=0.1, epsilon=1e-08 
[CV]  hidden_layer_sizes=(100, 100, 100), activation=tanh, solver=lbfs, alpha=0.01778279410038923, batch_size=10, learning_rate=invscaling, learning_rate_init=0.00042169650342858224, power_t=0.01, max_iter=74989, random_state=42, momentum=0.23713737056616552, nesterovs_momentum=Fal

...

[CV] hidden_layer_sizes=(100, 50), activation=relu, solver=sgd, alpha=0.00017782794100389227, batch_size=1, learning_rate=adaptive, learning_rate_init=0.21544346900318845, power_t=0.4216965034285822, max_iter=4216, random_state=42, momentum=0.31622776601683794, nesterovs_momentum=False, early_stopping=False, beta_1=0.01, beta_2=0.6799860450033224, epsilon=1e-08 
[CV]  hidden_layer_sizes=(100, 50), activation=relu, solver=sgd, alpha=0.00017782794100389227, batch_size=1, learning_rate=adaptive, learning_rate_init=0.21544346900318845, power_t=0.4216965034285822, max_iter=4216, random_state=42, momentum=0.31622776601683794, nesterovs_momentum=False, early_stopping=False, beta_1=0.01, beta_2=0.6799860450033224, epsilon=1e-08, score=0.5003335557038026, total=37.5min
[CV]  hidden_layer_sizes=(100, 100, 100), activation=relu, solver=sgd, alpha=0.0031622776601683794, batch_size=1, learning_rate=constant, learning_rate_init=0.0001, power_t=0.1333521432163324, max_iter=23713, random_state=42, mom

Process ForkPoolWorker-3:
Process ForkPoolWorker-8:
Process ForkPoolWorker-5:
Process ForkPoolWorker-6:
Process ForkPoolWorker-4:
Process ForkPoolWorker-2:
Process ForkPoolWorker-7:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/ljosfer/anaconda3/envs/deep/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ljosfer/anaconda3/envs/deep/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/ljosfer/anaconda3/envs/deep/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/ljosfer/anaconda3/envs/deep/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ljosfer/anaconda3/envs/deep/lib/python3.6/multiprocessing/process.py", line

KeyboardInterrupt: 

KeyboardInterrupt
  File "/home/ljosfer/anaconda3/envs/deep/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/ljosfer/anaconda3/envs/deep/lib/python3.6/multiprocessing/pool.py", line 108, in worker
    task = get()
  File "/home/ljosfer/anaconda3/envs/deep/lib/python3.6/multiprocessing/queues.py", line 334, in get
    with self._rlock:
  File "/home/ljosfer/anaconda3/envs/deep/lib/python3.6/multiprocessing/synchronize.py", line 95, in __enter__
    return self._semlock.__enter__()
KeyboardInterrupt


[CV]  hidden_layer_sizes=(100, 100, 100), activation=relu, solver=sgd, alpha=0.0031622776601683794, batch_size=1, learning_rate=constant, learning_rate_init=0.0001, power_t=0.1333521432163324, max_iter=23713, random_state=42, momentum=0.1333521432163324, nesterovs_momentum=True, early_stopping=False, beta_1=0.05, beta_2=0.26076542044591006, epsilon=1e-08, score=0.9859906604402935, total=21.3min


In [ ]:
best_indiv = LinearSVC(C = 5623.413251903491, 
                     penalty = 'l1',
                     loss = 'squared_hinge',
                     dual = False,
                     multi_class = 'ovr',
                     fit_intercept = True,
                     intercept_scaling = 0.03162277660168379,
                     random_state = 42,
                     tol = 1e-05,
                     max_iter = 316.22776601683796)